In [0]:
#1)- Importing key modules

In [0]:
# Let's be rebels and ignore warnings for now
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [0]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
import sys

In [4]:
! pip install turicreate

In [0]:
import turicreate as tc

In [0]:
# 2)-Loading Data

In [0]:
url = 'https://raw.githubusercontent.com/HassanSherwani/Product_Purchase_Frequency/master/20190207_transactions%20.json'

In [0]:
transactions = pd.read_json(url, lines= True)

In [0]:
# 3)-Exploring dataset

In [10]:
transactions.head()

,id,products
0,0,"[185, 30, 77, 188, 78, 125, 45, 155, 241, 229,..."
1,1,"[119, 148, 108, 34, 157, 82, 113, 45, 165]"
2,2,"[173, 103, 229, 240]"
3,3,[91]
4,4,"[175, 192, 54, 172]"


In [11]:
transactions.shape

(2500, 2)

In [12]:
transactions.info() # checking missing values

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2500 entries, 0 to 2499
Data columns (total 2 columns):
id          2500 non-null int64
products    2500 non-null object
dtypes: int64(1), object(1)
memory usage: 58.6+ KB


In [13]:
transactions.describe()

,id
count,2500.00000
mean,1249.50000
std,721.83216
min,0.00000
25%,624.75000
50%,1249.50000
75%,1874.25000
max,2499.00000


In [0]:
### Adding Features: Create a separate dataframe for recommending users
customers=transactions['id']

In [0]:
cust_2_rec=customers[:1000]

In [16]:
### break down each list of items in the products column into rows and count the number of products bought by a user

pd.melt(transactions.head(2).set_index('id')['products'].apply(pd.Series).reset_index(), 
             id_vars=['id'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['id', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})

,id,productId,purchase_count
0,0,30.0,1
1,0,45.0,1
2,0,77.0,1
3,0,78.0,1
4,0,89.0,2
5,0,125.0,1
6,0,133.0,1
7,0,155.0,1
8,0,161.0,1
9,0,185.0,1


In [0]:
# 4)- Making data for user, item, and target field

In [18]:
### 4a)-One for purchase count
s=time.time()

data = pd.melt(transactions.set_index('id')['products'].apply(pd.Series).reset_index(), 
             id_vars=['id'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['id', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
data['productId'] = data['productId'].astype(np.int64)

print("Execution time:", round((time.time()-s)/60,2), "minutes")

Execution time: 0.01 minutes


In [19]:
data.shape

(24811, 3)

In [20]:
data.head()

,id,productId,purchase_count
0,0,30,1
1,0,45,1
2,0,77,1
3,0,78,1
4,0,89,2


In [0]:
### 4-b)-Dummy as target 
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy

In [0]:
data_dummy = create_data_dummy(data)

In [23]:
data_dummy.head()

,id,productId,purchase_count,purchase_dummy
0,0,30,1,1
1,0,45,1,1
2,0,77,1,1
3,0,78,1,1
4,0,89,2,1


In [0]:
### 4-c)-Normalize item
"""
we normalize purchase frequency of each item across users by first creating a user-item matrix 

"""
df_matrix = pd.pivot_table(data, values='purchase_count', index='id', columns='productId')

In [25]:
df_matrix.head()

productId,1,2,3,4,5,6,7,8,9,10,...,241,242,243,244,245,246,247,248,249,250
id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
df_matrix.shape

(2378, 250)

In [27]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())

df_matrix_norm.head()

productId,1,2,3,4,5,6,7,8,9,10,...,241,242,243,244,245,246,247,248,249,250
id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
print(df_matrix_norm.shape)

(2378, 250)


In [0]:
### 4-d)- create a table for input to the modeling

d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['id'], value_name='scaled_purchase_freq').dropna()

In [30]:
data_norm.head()

,id,productId,scaled_purchase_freq
11,12,1,0.0
38,41,1,0.0
40,43,1,0.0
93,96,1,0.0
100,103,1,0.0


In [31]:
data_norm.shape

(22530, 3)

In [0]:
###4-e) a function for normalizing data
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='id', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['id'], value_name='scaled_purchase_freq').dropna()

In [0]:
# 5)-Split train and test set

In [34]:
train, test = train_test_split(data, test_size = .2)
print(train.shape, test.shape)

(19848, 3) (4963, 3)


In [0]:
"""convert dataframe to SFrame """
train_data = tc.SFrame(train)
test_data = tc.SFrame(test)

In [36]:
train_data

id,productId,purchase_count
1840,170,1
2317,13,1
1438,17,1
1237,233,1
1264,16,1
2035,113,1
228,66,1
1381,164,1
412,130,1
2337,199,1


In [37]:
test_data

id,productId,purchase_count
206,64,1
218,20,1
2071,39,1
2168,31,1
1392,140,1
57,180,1
1981,105,1
750,145,1
331,6,1
1416,218,1


In [0]:
### 5.2)- Define a split_data function for splitting data to training and test set
def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [0]:
### 5.3)-Apply for both dummy table and scaled/normalized purchase table

In [0]:
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

In [0]:
#6)-Model

In [0]:
### 6.1)- Parameters to define field names for purchase count as target feature
user_id = 'id'
item_id = 'productId'
users_to_recommend=list(cust_2_rec)
target = 'purchase_count'
n_rec = 10 # number of items to recommend
n_display = 30 # 1st 30 rows to display

In [43]:
popularity_model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 19848 observations with 2354 users and 250 items.

Data prepared in: 0.039691s

19848 observations to process; with 250 unique items.

In [44]:
popularity_recomm = popularity_model.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm.print_rows(n_display)

recommendations finished on 1000/1000 queries. users per second: 73817.1

+----+-----------+--------------------+------+
| id | productId |       score        | rank |
+----+-----------+--------------------+------+
| 0  |    207    | 1.0963855421686748 |  1   |
| 0  |     96    | 1.0921052631578947 |  2   |
| 0  |    201    | 1.0895522388059702 |  3   |
| 0  |    153    | 1.0769230769230769 |  4   |
| 0  |     63    | 1.0689655172413792 |  5   |
| 0  |    150    | 1.065934065934066  |  6   |
| 0  |     26    | 1.0632911392405062 |  7   |
| 0  |    131    | 1.0609756097560976 |  8   |
| 0  |    186    | 1.0579710144927537 |  9   |
| 0  |    102    | 1.0574712643678161 |  10  |
| 1  |    207    | 1.0963855421686748 |  1   |
| 1  |     96    | 1.0921052631578947 |  2   |
| 1  |    201    | 1.0895522388059702 |  3   |
| 1  |    153    | 1.0769230769230769 |  4   |
| 1  |     63    | 1.0689655172413792 |  5   |
| 1  |    150    | 1.065934065934066  |  6   |
| 1  |     26    | 1.0632911392405062 |  7   |
| 1  |    131    | 1.0609756097560976 |  8   |
| 1  |    186

In [45]:
"""
Through this model, we predicted the recommendation items using scores by popularity. As you can tell for each model results above, the rows show the first 30 records from 1000 users
with 10 recommendations. These 30 records include 3 users and their recommended items, along with score and descending ranks.

"""

'\nThrough this model, we predicted the recommendation items using scores by popularity. As you can tell for each model results above, the rows show the first 30 records from 1000 users\nwith 10 recommendations. These 30 records include 3 users and their recommended items, along with score and descending ranks.\n\n'

In [46]:
# Checking most frequent items  
train.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

productId
207    1.096386
96     1.092105
201    1.089552
153    1.076923
63     1.068966
150    1.065934
26     1.063291
131    1.060976
186    1.057971
102    1.057471
49     1.057143
44     1.056180
181    1.055556
129    1.055556
85     1.054945
122    1.054348
88     1.054054
152    1.052632
50     1.052083
117    1.051948
Name: purchase_count, dtype: float64

In [47]:
"""
products 96, 201, 125,112 and 26 are the most popular (best-selling) across customers.


"""

'\nproducts 96, 201, 125,112 and 26 are the most popular (best-selling) across customers.\n\n\n'

In [48]:
"""



# Define a model
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model
    
 """

"\n\n\n\n# Define a model\ndef model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):\n    if name == 'popularity':\n        model = tc.popularity_recommender.create(train_data, \n                                                    user_id=user_id, \n                                                    item_id=item_id, \n                                                    target=target)\n    recom = model.recommend(users=users_to_recommend, k=n_rec)\n    recom.print_rows(n_display)\n    return model\n    \n "

In [0]:
### 6.2)- purchase dummy as target feature
user_id = 'id'
item_id = 'productId'
users_to_recommend=list(cust_2_rec)
target = 'purchase_dummy'
n_rec = 10 # number of items to recommend
n_display = 30 # 1st 30 rows to display

In [50]:
popularity_model_dummy = tc.popularity_recommender.create(train_data_dummy, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 19848 observations with 2357 users and 250 items.

Data prepared in: 0.039141s

19848 observations to process; with 250 unique items.

In [51]:
popularity_recomm_4_dummy = popularity_model_dummy.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm_4_dummy.print_rows(n_display)

recommendations finished on 1000/1000 queries. users per second: 127356

+----+-----------+-------+------+
| id | productId | score | rank |
+----+-----------+-------+------+
| 0  |    240    |  1.0  |  1   |
| 0  |     99    |  1.0  |  2   |
| 0  |    249    |  1.0  |  3   |
| 0  |     7     |  1.0  |  4   |
| 0  |    198    |  1.0  |  5   |
| 0  |     12    |  1.0  |  6   |
| 0  |    200    |  1.0  |  7   |
| 0  |    111    |  1.0  |  8   |
| 0  |    184    |  1.0  |  9   |
| 0  |    205    |  1.0  |  10  |
| 1  |     99    |  1.0  |  1   |
| 1  |    249    |  1.0  |  2   |
| 1  |     7     |  1.0  |  3   |
| 1  |    198    |  1.0  |  4   |
| 1  |     12    |  1.0  |  5   |
| 1  |    200    |  1.0  |  6   |
| 1  |    111    |  1.0  |  7   |
| 1  |    184    |  1.0  |  8   |
| 1  |    205    |  1.0  |  9   |
| 1  |    125    |  1.0  |  10  |
| 2  |     99    |  1.0  |  1   |
| 2  |    249    |  1.0  |  2   |
| 2  |     7     |  1.0  |  3   |
| 2  |    198    |  1.0  |  4   |
| 2  |     12    |  1.0  |  5   |
| 2  |    200    |  1.0  |  6   |
| 2  |    111 

In [0]:
### 6.3)- Applying 'scaled_purchase_freq' as target feature on model
user_id = 'id'
item_id = 'productId'
users_to_recommend=list(cust_2_rec)
target = 'scaled_purchase_freq'
n_rec = 10 # number of items to recommend
n_display = 30 # 1st 30 rows to display

In [53]:
popularity_model_scaled = tc.popularity_recommender.create(train_data_norm, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 18024 observations with 2339 users and 227 items.

Data prepared in: 0.032906s

18024 observations to process; with 227 unique items.

In [54]:
popularity_recomm_4_scaled = popularity_model_scaled.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm_4_scaled.print_rows(n_display)

recommendations finished on 1000/1000 queries. users per second: 89766.6

+----+-----------+----------------------+------+
| id | productId |        score         | rank |
+----+-----------+----------------------+------+
| 0  |     96    |        0.0875        |  1   |
| 0  |     26    |         0.08         |  2   |
| 0  |    153    |  0.0759493670886076  |  3   |
| 0  |    207    | 0.07142857142857142  |  4   |
| 0  |    201    | 0.07017543859649122  |  5   |
| 0  |    150    | 0.06896551724137931  |  6   |
| 0  |    117    | 0.06578947368421052  |  7   |
| 0  |    170    |  0.0641025641025641  |  8   |
| 0  |     49    | 0.06060606060606061  |  9   |
| 0  |    120    | 0.060240963855421686 |  10  |
| 1  |     96    |        0.0875        |  1   |
| 1  |     26    |         0.08         |  2   |
| 1  |    153    |  0.0759493670886076  |  3   |
| 1  |    207    | 0.07142857142857142  |  4   |
| 1  |    201    | 0.07017543859649122  |  5   |
| 1  |    150    | 0.06896551724137931  |  6   |
| 1  |    117    | 0.06578947368421052  |  7   |
| 1  |    170    |  

In [0]:
# 7)- Evaluate

In [56]:
test.head()

,id,productId,purchase_count
1889,206,64,1
2000,218,20,1
20484,2071,39,1
21436,2168,31,1
13878,1392,140,1


In [0]:
models_counts = [popularity_model]

In [0]:
model_names=['Popularity Model on Purchase Counts']

In [59]:
eval_counts = tc.recommender.util.compare_models(test_data, models_counts, model_names)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/1895 queries. users per second: 54359.6


Precision and recall summary statistics by cutoff
+--------+----------------------+----------------------+
| cutoff |    mean_precision    |     mean_recall      |
+--------+----------------------+----------------------+
|   1    | 0.011081794195250662 | 0.004586631486367625 |
|   2    | 0.012137203166226908 | 0.01032814005109519  |
|   3    | 0.01143359718557606  | 0.013815387192695915 |
|   4    | 0.011345646437994739 | 0.018393851823930985 |
|   5    | 0.011503957783641177 | 0.023517611090170448 |
|   6    | 0.011697449428320164 | 0.029381413075344438 |
|   7    | 0.011458725970599314 | 0.033493738744398344 |
|   8    | 0.011477572559366756 | 0.037917661347740515 |
|   9    | 0.011668132512459703 | 0.04277254261423132  |
|   10   | 0.011715039577836404 | 0.047759978221719665 |
+--------+----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.14893140539549665

Per User RMSE (best)
+------+------+-------+
|  id  | rmse | count |
+------+------+-------+